In [ ]:
import geopandas
import urllib
import json
import pandas
from datetime import date, timedelta
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import shapely.ops



In [ ]:
import fiona;
fiona.supported_drivers


In [ ]:
mapping_crs = "EPSG:3347" # NAD83 / Statistics Canada Lambert
# mapping_crs = "EPSG:3857" # Pseudo-Mercator
output_crs = "EPSG:4326" # Lat/Lng https://epsg.io/4326

In [ ]:
def load_geodataset(remote_path, local_path, simplify=None, refresh_cache=False):
    df = None
    if not refresh_cache:
        try:
            df = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if df is None:
        print(f"Loading {remote_path}")
        df = geopandas.read_file(remote_path)
        if simplify:
            df["geometry"] = df["geometry"].simplify(simplify)

        print(f"Saving to {local_path}")
        df.to_file(local_path)

    return df

In [ ]:
def get_roads(refresh_cache=False):
    remote_path = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/RNF-FRR/files-fichiers/lrnf000r21a_e.zip"
    local_path = "data/roads.shp"
    return load_geodataset(remote_path, local_path, refresh_cache=refresh_cache)


In [ ]:
def get_highways(refresh_cache=False, columns=["NAME", "TYPE", "CSDNAME_R", "PRNAME_L", "geometry"]):
    local_path = "data/highways.shp"
    
    highways = None
    if not refresh_cache:
        try:
            highways = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if highways is None:
        roads = get_roads(refresh_cache)
        highways = roads.query('RANK <= "2"')
        print(f"Saving to {local_path}")
        highways.to_file(local_path)

    return highways[columns].to_crs(mapping_crs)


In [ ]:
# this is very slow the first time you run it (downloads a 296 MB zipfile), but it saves a smaller local version for future runs (139 MB)
highways = get_highways()
highways

In [ ]:
highways.plot()

In [ ]:
def get_provinces(refresh_cache=False):
    remote_path = "https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/files-fichiers/2016/lpr_000b16a_e.zip"
    local_path = "data/provinces.shp"
    # simplify=1000 turns a 54 MB file to a 3 MB file
    return load_geodataset(remote_path, local_path, simplify=1000, refresh_cache=refresh_cache)

In [ ]:
provinces = get_provinces()
provinces.plot()

In [ ]:
# connector_type: [all, NEMA1450, NEMA515, NEMA520, J1772, J1772COMBO, CHADEMO, TESLA]
def get_stations(connector_type="J1772COMBO", country="CA", refresh_cache=False):
    remote_path=f"https://developer.nrel.gov/api/alt-fuel-stations/v1.geojson?api_key=JDgwj9XoVgfg8ryEgz1AOnnBbfoLATWjy4x6dOlv&country={country}&owner_type=all&cards_accepted=all&offset=0&fuel_type=ELEC&access=public&status=E&ev_charging_level=dc_fast&ev_connector_type={connector_type}&ev_network=all"
    local_path=f"data/stations_{country}_{connector_type}.shp"

    stations = None
    if not refresh_cache:
        try:
            stations = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if stations is None:
        print(f"Loading {remote_path}")
        stations = geopandas.read_file(remote_path)
        stations.crs = "EPSG:4326"
        columns=["id", "open_date", "status_code", "station_name",
           "city", "state", "street_address", "ev_network", "geometry"]
        stations = stations[columns]
        print(f"Saving to {local_path}")
        stations.to_file(local_path)

    return stations.to_crs(mapping_crs)

    
    

In [ ]:
# Hyudani Kona - CCS Combo - 415 km
# Hyudani Ioniq 5 - CCS Combo - 354 km
# Hyudani Ioniq 5 Long Range - CCS Combo - 488 km
# Nissan Leaf - Chademo 349 km
# Telsa Model 3 Standard Range - 401 km
# Tesla Model 3 Extended Range - 518 km

connector_type= "J1772COMBO" # one of ["CHADEMO", "J1772COMBO", "TESLA"]
max_range = 415

In [ ]:
stations = get_stations(connector_type)
stations

In [ ]:
stations.to_crs(output_crs).to_file(f"maps/stations_map_{connector_type}.geojson")

In [ ]:
stations["open_date"] = stations["open_date"].apply(lambda x: (date.fromisoformat(x) if x is not None else date.today()))


In [ ]:
stations.plot()

In [ ]:
ax = stations.plot(color='g', markersize=10)
highways.plot(ax=ax)

In [ ]:
# Method 1 for finding close stations - keep adding adjacent segments next to each EV
def find_stations_old_method():
    close_stations = geopandas.sjoin_nearest(highways, stations[["geometry"]], how='left', distance_col='charger_distance', max_distance=5000)
    close_stations = close_stations.drop(['index_right'], axis=1)

    found = close_stations[close_stations.charger_distance >= 0]
    not_found = close_stations[close_stations.charger_distance.isna()]
    f"{len(found)} / {len(not_found)}"

    changes = True
    tolerance = 0.1
    i = 0

    while changes and len(not_found) > 0:
        adjacent = geopandas.sjoin_nearest(not_found.drop(["charger_distance"], axis=1), found[["geometry", "charger_distance"]], how='left', distance_col='closest_distance', max_distance=tolerance)
        adjacent["charger_distance"] = adjacent["charger_distance"] + adjacent.geometry.length + adjacent["closest_distance"]
        adjacent = adjacent[found.columns]
        changes = len(adjacent[adjacent.charger_distance >= 0]) > 0
        found = pandas.concat([found, adjacent[adjacent.charger_distance >= 0]])
        not_found = adjacent[adjacent.charger_distance.isna()]
        i += 1
        if i % 100 == 0:
            print(f"{len(found)} / {len(not_found)}")
    print(f"{len(found)} / {len(not_found)}")

    not_found["charger_distance"] = 999999
    found = pandas.concat([found, not_found])
    return found

# found = find_stations_old_method()

In [ ]:
# Method 2 for finding close stations - look at adjacent stations and find minimum path to a charger
def calculate_ev_highways(stations, highways, ev_highways=None):
    close_stations = geopandas.sjoin_nearest(highways, stations[["geometry"]], how='left', distance_col='charger_distance', max_distance=5000)
    close_stations = close_stations.drop(['index_right'], axis=1)

    if ev_highways is None:
        max_distance = 499999 # We don't care about roads more than 500 kms away from a station
        ev_highways = highways.copy()
        ev_highways["charger_distance"] = max_distance
    
    next_round = close_stations[close_stations["charger_distance"] >= 0]
    ev_highways.loc[next_round.index, "charger_distance"] = next_round["charger_distance"]

    changes = True
    tolerance = 0.1
    i = 0

    while changes:
        adjacent = geopandas.sjoin_nearest(ev_highways[["geometry", "charger_distance"]], next_round, how='left', distance_col="closest_distance", max_distance=tolerance)
        adjacent = adjacent[~adjacent["closest_distance"].isna()]
        adjacent["charger_distance"] = adjacent["charger_distance_right"] + adjacent.geometry.length + adjacent["closest_distance"]
        # Limit adjacents to shortest distance to a charger per segment
        adjacent.sort_values('charger_distance', inplace=True)
        adjacent = adjacent[~adjacent.index.duplicated()]
        # Only consider segments that are now shorter
        next_round = adjacent[adjacent["charger_distance"] < adjacent["charger_distance_left"] ]
        next_round = next_round[ev_highways.columns]
        ev_highways.loc[next_round.index, "charger_distance"] = next_round["charger_distance"]
        
        changes = len(next_round) > 0
        sum_distance = ev_highways["charger_distance"].sum()
        i += 1
        print(f'{i} - changed: {len(next_round)} distances, sum distance={sum_distance}', end='\r')

    return ev_highways



In [ ]:
def categorize_distance(dist):
    if dist * 2 <= min(50 * 1000, max_range * 1000 * 0.8 * 0.25):
        return "1 - Great"
    if dist * 2 <= min(100 * 1000, max_range * 1000 * 0.8 * 0.5):
        return "2 - Good"
    if dist * 2 <= min(200 * 1000, max_range * 1000 * 0.8 * 0.75):
        return "3 - Doable"
    if dist * 2 <= min(300 * 1000, max_range * 1000 * 0.8):
        return "4 - Hard"
    if dist * 2 <= min(350 * 1000, max_range * 1000 * 0.85):
        return "5 - Risky"
    return "6 - Inaccessible"

In [ ]:
def draw_map(filtered_stations,
             ev_highways,
             station_count_df,
             as_of=None,
             provinces=provinces,
             region='CA',
             highway_options={'legend': False, 'cmap': 'YlGnBu_r'},
             station_options={'markersize':0.7, 'color': 'xkcd:dark blue'},
             dist_histogram_options={
                 'show': True,
                 'inset_axes': [0.75, 0.8, 0.25, 0.2],
                 'xlim': (0, 40000),
                 'xticks': [0, 10000, 20000, 30000],
                 'unit': 'km',
                 'x_label_position': 'bottom',
             },
             station_count_options={
                 'show': True,
                 'ylim': (0, 2000),
                 'inset_axes': [0.045, 0.03, 0.3, 0.15],
             },
             filename=None,
             figsize=(12.5,11),
             title='Canadian Highways with EV Fast Charging'
    ):
    ev_highways = ev_highways.copy()
    ev_highways['distance_cat'] = ev_highways['charger_distance'].apply(categorize_distance)
    
    ax = ev_highways.plot(column='distance_cat', figsize=figsize, **highway_options)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title(title)

    cmap = matplotlib.cm.YlGnBu_r

    if dist_histogram_options.get('show', False):
        axin1 = ax.inset_axes(dist_histogram_options.get('inset_axes'))

        ev_highways['length_km'] = ev_highways.length / 1000
        dist_table = ev_highways.groupby(['distance_cat'])['length_km'].sum()
        distance_histogram = dist_table.plot(ax=axin1, kind='barh', color=[cmap(0), cmap(1/5), cmap(2/5), cmap(3/5), cmap(4/5), cmap(5/5)], edgecolor='gray')
        distance_histogram.invert_yaxis()
        distance_histogram.set_xlim(dist_histogram_options.get('xlim'))
        distance_histogram.set_xticks(dist_histogram_options.get('xticks'))
        distance_histogram.set_xlabel(dist_histogram_options.get('unit'))
        axin1.xaxis.set_label_position(dist_histogram_options.get('x_label_position')) 
        if dist_histogram_options.get('x_label_position') == 'top':
            axin1.xaxis.tick_top()
        distance_histogram.set_ylabel('')
        category_lables = ['Great', 'Good', 'Doable', 'Hard', 'Risky', 'Inaccessible']
        distance_histogram.yaxis.set_major_formatter(lambda x, y: category_lables[x])

    if station_count_df is not None and as_of is not None and station_count_options.get('show', False):
        axin2 = ax.inset_axes(station_count_options.get('inset_axes'))
        station_counts = station_count_df.copy()
        station_counts.loc[station_counts.index.date > as_of, 'count'] = np.nan
        station_count_chart = station_counts.plot(ax=axin2, legend=False)
        station_count_chart.set_title(f'{as_of.strftime("%Y %b")} - {len(filtered_stations)} stations', {'family': 'monospace'})
        station_count_chart.set_ylim(station_count_options.get('ylim'))

    provinces.plot(ax=ax, figsize=figsize, color='xkcd:cool gray', zorder=0)
    provinces.boundary.plot(ax=ax, figsize=figsize, color='gray', linewidth=0.3, zorder=1)
    
    plot = filtered_stations.plot(ax=ax, figsize=figsize, **station_options)
    plt.tight_layout()
    if filename is None:
        filename = f'monthly_ev_highways_{region}_{connector_type}_{max_range}_{end_of_month.strftime("%Y-%m")}.png'
    plot.get_figure().savefig(f'output/{filename}')
    
    plt.close('all')


In [ ]:
ev_highways = calculate_ev_highways(stations, highways)


In [ ]:
draw_map(stations, ev_highways, None, filename=f'ev_highways_CA_{connector_type}_{max_range}_today.png')


In [ ]:
def export_ev_highways(ev_highways, country, connector_type):
    ev_highways = ev_highways.copy()
    ev_highways['rank'] = ev_highways['charger_distance'].apply(categorize_distance)
    ev_highways['dist_km'] = ev_highways['charger_distance'].apply(lambda x: int(round(x / 1000)))
    ev_highways.drop(columns=['charger_distance'], inplace=True)
    # Simplify before exporting
    ev_highways['geometry'] = ev_highways['geometry'].simplify(10)

    ev_highways.to_crs(output_crs).to_file(f'maps/ev_highways_{country}_{connector_type}.geojson')



In [ ]:
export_ev_highways(ev_highways.drop(columns=['TYPE', 'PRNAME_L']).rename(columns={'NAME': 'Name', 'CSDNAME_R': 'Place'}), 'CA', connector_type)

In [ ]:
def next_month(d):
    return date(d.year, d.month + 1, 1) if d.month < 12 else date(d.year + 1, 1, 1)

In [ ]:
def calculate_station_count_df(stations):
    stations = stations.copy()

    start_date = stations["open_date"].min()
    end_date = date.today()

    return pandas.DataFrame({"count": stations.groupby("open_date")["id"].count().reindex(pandas.date_range(start_date, end_date, freq='D'), fill_value=0).cumsum()})

station_count_df = calculate_station_count_df(stations)
station_count_df = station_count_df[station_count_df.index >= '2015-01-01']
station_count_df.plot()

In [ ]:
start_of_month = date(2015, 1, 1)

ev_highways = None
while start_of_month <= date.today():
    end_of_month = next_month(start_of_month) - timedelta(days=1)
    print(end_of_month.strftime("%Y-%m-%d"))

    filtered_stations = stations[stations["open_date"] <= end_of_month]
    
    ev_highways = calculate_ev_highways(filtered_stations, highways, ev_highways)

    draw_map(filtered_stations, ev_highways, station_count_df, end_of_month)
    
    start_of_month = next_month(start_of_month)
    print()


In [ ]:
def province_good_and_great(ev_highways):
    ev_highways = ev_highways.copy()
    ev_highways['distance_cat'] = ev_highways['charger_distance'].apply(categorize_distance)
    ev_highways['length_km'] = ev_highways.length / 1000
    
    good_by_province = ev_highways.groupby(["PRNAME_L", "distance_cat"])["length_km"].sum()
    
    by_prov = pandas.DataFrame(good_by_province).reset_index(level=1).pivot_table(index=["PRNAME_L"], columns="distance_cat", values="length_km").fillna(0)
    by_prov["good_%"] = (by_prov["1 - Great"] + by_prov["2 - Good"]) / (by_prov["1 - Great"] + by_prov["2 - Good"] + by_prov["3 - Doable"] + by_prov["4 - Hard"] + by_prov["5 - Risky"] + by_prov["6 - Inaccessible"])
    return pandas.merge(provinces, by_prov, how="left", left_on=["PRNAME"], right_on=["PRNAME_L"]).fillna(0)


In [ ]:
by_prov = province_good_and_great(ev_highways)
by_prov


In [ ]:
ev_highways["good"] = ev_highways["charger_distance"] < 50*1000
ev_highways

In [ ]:
ax = by_prov.plot(column="good_%", cmap="Greens", edgecolor="black", linewidth=0.1, figsize=(12.5,11))
ax.set_title("Percentage of Major Highways With Good EV Fast Charging By Province")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

xy_overwrites = {
    "P.E.I.": (8351561, 1680000),
    "N.S.": (8650000, 1460000),
    "N.B.": (8050000, 1290000),
    "Nvt.": (6100000, 3200000),
}

def annotate_province_percentages(x):
    if x["PREABBR"] == 'Nvt.':
        # skip as there are no major highways
        return
    xy = xy_overwrites.get(x["PREABBR"], x.geometry.centroid.coords[0])
    return ax.annotate(text=f"{x['good_%']:.0%}", xy=xy, ha="center", fontsize=14)
by_prov.apply(annotate_province_percentages, axis=1);
ev_highways[ev_highways["good"] == True].plot(ax=ax, linewidth=0.2, color="xkcd:electric blue")
plot = ev_highways[ev_highways["good"] == False].plot(ax=ax, linewidth=0.2, color="xkcd:light grey")
plt.tight_layout()
plot.get_figure().savefig(f"output/ev_highways_CA_{connector_type}_good.png")
plot

In [ ]:
# new_stations
# Yorkton


new_stations_df = pandas.DataFrame({
    'City': ['Yorkton', 'Grande Prairie', 'Whitecourt', 'Boyle', 'Minnedosa', 'Prince Albert', 'McLeod Lake', 'Hanna', 'Kindersley', 'Winnard'],
    'Province': ['SK', 'AB', 'AB', 'AB', 'MB', 'SK', 'BC', 'AB', 'SK', 'SK'],
    'Latitude': [51.210516, 55.171337, 54.137583, 54.587165, 50.246068, 53.199422, 54.991843, 51.634314, 51.474346, 51.7687349],
    'Longitude': [-102.448397, -118.821136, -115.686344, -112.803389, -99.839088, -105.759733, -123.034198, -111.942042, -109.168040, -104.1791185],
    'Population': [16343, 63166, 9721, 845, 2449, 35926, 94, 2559, 4597, 1732],
})

new_stations = geopandas.GeoDataFrame(new_stations_df, geometry=geopandas.points_from_xy(new_stations_df.Longitude, new_stations_df.Latitude))
new_stations.crs = "EPSG:4326"
new_stations = new_stations.to_crs(mapping_crs)
new_stations

In [ ]:
fantasy_stations = pandas.concat([stations[["geometry"]], new_stations[["geometry"]]])
fantasy_ev_highways = calculate_ev_highways(fantasy_stations, highways, ev_highways)


In [ ]:
draw_map(new_stations, fantasy_ev_highways, None, filename=f'ev_highways_{connector_type}_{max_range}_fantasy.png', station_options={"markersize":15.0, "color": "xkcd:coral"}, title="Fantasy EV Fast Charging (10 Extra Stations)")

In [ ]:
# US Map

In [ ]:
# Switch to a better projection for the US
mapping_crs = "EPSG:3857" # Pseudo-Mercator


In [ ]:
def get_us_highways(refresh_cache=False):
    local_path = "data/us_highways.shp"
    remote_path = "https://www2.census.gov/geo/tiger/TIGER2022/PRIMARYROADS/tl_2022_us_primaryroads.zip"
    return load_geodataset(remote_path, local_path, refresh_cache=refresh_cache).to_crs(mapping_crs)


In [ ]:
def load_wikipedia_map(map_title):
    url = f'https://en.wikipedia.org/w/api.php?format=json&formatversion=2&action=jsondata&title={urllib.parse.quote(map_title)}'
    print(url)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return geopandas.GeoDataFrame.from_features(data["jsondata"]["data"])


In [ ]:
# The map here is nicer than the Census one
# https://en.wikipedia.org/wiki/List_of_Interstate_Highways#/map/0
def get_us_highways_from_wikipedia(refresh_cache=False):
    local_path = "data/us_highways_wikipedia.shp"
    remote_maps = [
        'Interstate 2.map',
        'Interstate 4.map',
        'Interstate 5.map',
        'Interstate 8.map',
        'Interstate 10.map',
        'Interstate 11.map',
        'Interstate 12.map',
        'Interstate 14.map',
        'Interstate 15.map',
        'Interstate 16.map',
        'Interstate 17.map',
        'Interstate 19.map',
        'Interstate 20.map',
        'Interstate 22.map',
        'Interstate 24.map',
        'Interstate 25.map',
        'Interstate 26.map',
        'Interstate 27.map',
        'Interstate 29.map',
        'Interstate 30.map',
        'Interstate 35.map',
        'Interstate 37.map',
        'Interstate 39.map',
        'Interstate 40.map',
        'Interstate 41.map',
        'Interstate 43.map',
        'Interstate 44.map',
        'Interstate 45.map',
        'Interstate 49 1.map',
        'Interstate 55.map',
        'Interstate 57.map',
        'Interstate 59.map',
        'Interstate 64.map',
        'Interstate 65.map',
        'Interstate 66.map',
        'Interstate 68.map',
        'Interstate 69.map',
        'Interstate 70.map',
        'Interstate 71.map',
        'Interstate 72.map',
        'Interstate 73.map',
        'Interstate 74.map',
        'Interstate 75.map',
        'Interstate 76 (Ohio–New Jersey).map',
        'Interstate 76 (Colorado–Nebraska).map',
        'Interstate 77.map',
        'Interstate 78.map',
        'Interstate 79.map',
        'Interstate 80.map',
        'Interstate 81.map',
        'Interstate 82.map',
        'Interstate 83.map',
        'Interstate 84 (Oregon–Utah).map',
        'Interstate 84 (Pennsylvania–Massachusetts).map',
        'Interstate 85.map',
        'Interstate 86 (Idaho).map',
        'Interstate 86 (Pennsylvania–New York).map',
        'Interstate 87 (North Carolina).map',
        'Interstate 87 (New York).map',
        'Interstate 88 (Illinois).map',
        'Interstate 88 (New York).map',
        'Interstate 89.map',
        'Interstate 90.map',
        'Interstate 91.map',
        'Interstate 93.map',
        'Interstate 94.map',
        'Interstate 95.map',
        'Interstate 96.map',
        'Interstate 97.map',
        'Interstate 99.map',
        'Interstate 35W (Texas).map',
        'Interstate 35W (Minnesota).map',
    ]
    
    df = None
    if not refresh_cache:
        try:
            df = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if df is None:
        print(f"Loading remote files")
        df = pandas.concat([load_wikipedia_map(remote_map) for remote_map in remote_maps])
        df.crs = "EPSG:4326"

        # Cleanup the data
        df.reset_index(inplace=True)
        df.loc[df["name"].isna(), "name"] = df["Name"]
        df = df[["name", "geometry"]]

        print(f"Saving to {local_path}")
        df.to_file(local_path)
            
    return df.to_crs(mapping_crs)
    

In [ ]:
us_highways = get_us_highways_from_wikipedia()

In [ ]:
# Explode highway segments into 5 km chunks (they're too long by default)
def split_highways(highways, max_length=5000):
    def split_geometry(geom, max_distance=5000):
        return [shapely.ops.substring(geom, i, i + max_distance) for i in range(0, int(geom.length), max_distance)]

    us_highways['split_geometry'] = us_highways['geometry'].apply(split_geometry)
    split_df = geopandas.GeoDataFrame([{'name': tup.name, 'geometry': geom} for tup in us_highways.itertuples() for geom in tup.split_geometry])
    split_df.crs = highways.crs
    return split_df

In [ ]:
us_highways = split_highways(us_highways)

In [ ]:
us_highways.plot()

In [ ]:
def get_states(refresh_cache=False):
    remote_path = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip"
    local_path = "data/states.shp"
    return load_geodataset(remote_path, local_path, refresh_cache=refresh_cache).to_crs(mapping_crs)

In [ ]:
states = get_states()

In [ ]:
# connector_type = 'TESLA'

In [ ]:
us_stations = get_stations(connector_type, country="US")
us_stations

In [ ]:
us_stations.to_crs(output_crs).to_file(f"maps/stations_map_US_{connector_type}.geojson")

In [ ]:
us_stations["open_date"] = us_stations["open_date"].apply(lambda x: (date.fromisoformat(x) if x is not None else date.today()))

In [ ]:
ax = us_stations.plot(color='g', markersize=10)
us_highways.plot(ax=ax)

In [ ]:
us_ev_highways = calculate_ev_highways(us_stations, us_highways)


In [ ]:
us_ev_highways.plot(column='charger_distance')

In [ ]:
export_ev_highways(us_ev_highways, 'US', connector_type)

In [ ]:
non_continental_states = ["AK", "HI", "PR"]

In [ ]:
continental_states = states[~states["STUSPS"].isin(non_continental_states)]
continental_states.plot()

In [ ]:
def interstate_good_and_great(ev_highways):
    ev_highways = ev_highways.copy()
    ev_highways['distance_cat'] = ev_highways['charger_distance'].apply(categorize_distance)
    ev_highways['length_km'] = ev_highways.length / 1000

    good_groupby = ev_highways.groupby(["name", "distance_cat"])["length_km"].sum()
    by_name = pandas.DataFrame(good_groupby).reset_index(level=1).pivot_table(index=["name"], columns="distance_cat", values="length_km").fillna(0)
    by_name["good_%"] = (by_name["1 - Great"] + by_name["2 - Good"]) / (by_name["1 - Great"] + by_name["2 - Good"] + by_name["3 - Doable"] + by_name["4 - Hard"] + by_name["5 - Risky"] + by_name["6 - Inaccessible"])
    return by_name

interstate_summary = interstate_good_and_great(us_ev_highways)
pandas.set_option('display.max_rows', 100)
interstate_summary


In [ ]:
continental_us_stations = us_stations[(~us_stations["state"].isin(non_continental_states)) & (~us_stations["state"].isna())]
continental_us_stations.plot()

In [ ]:
draw_map(
    continental_us_stations,
    us_ev_highways,
    None,
    provinces=continental_states,
    dist_histogram_options={'show': False},
    station_count_options={'show':False},
    figsize=(15,8.5),
    title='Interstate Highways with EV Fast Charging Map',
    filename=f'ev_highways_US_{connector_type}_today.png'
)


In [ ]:
us_station_count_df = calculate_station_count_df(us_stations)
us_station_count_df = us_station_count_df[us_station_count_df.index >= '2015-01-01']
us_station_count_df.plot()

In [ ]:
start_of_month = date(2015, 1, 1)

us_ev_highways = None
while start_of_month <= date.today():
    end_of_month = next_month(start_of_month) - timedelta(days=1)
    print(end_of_month.strftime("%Y-%m-%d"))
    
    filtered_stations = continental_us_stations[continental_us_stations["open_date"] <= end_of_month]
    
    us_ev_highways = calculate_ev_highways(filtered_stations, us_highways, us_ev_highways)

    draw_map(filtered_stations,
             us_ev_highways,
             us_station_count_df,
             end_of_month,
             provinces=continental_states,
             figsize=(15,8.5),
             title='Interstate Highways with EV Fast Charging',
             region = 'US',
             dist_histogram_options={
                 'show': True,
                 'inset_axes': [0.80, 0.0, 0.20, 0.25],
                 'xlim': (0, 50000),
                 'xticks': range(0, 50000, 20000),
                 'unit': 'miles',
                 'x_label_position': 'top',
             },
             station_count_options={
                 'show': True,
                 'ylim': (0, 8000),
                 'inset_axes': [0.045, 0.036, 0.3, 0.15],
             },
    )
    
    start_of_month = next_month(start_of_month)
    print()